In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torch.nn.functional as F
import sys
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
GIT_ROOT = Path("../..").resolve()
print(GIT_ROOT)
SRC = GIT_ROOT / "src"
if not SRC in sys.path:
    sys.path.append(str(SRC))

/home/ubuntu/KEX---CT-reconstruction


In [2]:
import torch
import matplotlib.pyplot as plt
from utils.data import get_htc2022_train_phantoms, get_kits_train_phantoms
from utils.polynomials import Chebyshev, Legendre
from utils.tools import MSE
from geometries import HTC2022_GEOMETRY, CDTYPE, get_moment_mask, DEVICE


##CONFIG
geometry = HTC2022_GEOMETRY
ar = 0.25
M, K = 64, 64
PolynomialFamily = Legendre
##

In [4]:

##data

kits = torch.load(GIT_ROOT / "data/kits_phantoms_256.pt")
N, c, h, w = kits.shape
print(kits.shape)
kits = kits.reshape((N*c,h,w))
resized = torchvision.transforms.functional.resize(kits, (512, 512))
print(resized.shape)

plt.imshow(resized[700].cpu())

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [6]:
sinos = geometry.project_forward(resized)
print("sinos computed")
la_sinos, knwon_angles = geometry.zero_cropp_sinos(sinos, ar, 0)
n_known_betas = geometry.n_known_projections(ar)
batch_size = la_sinos.shape[0]

n_iters = 30
cn = torch.zeros((batch_size, M, K), device=DEVICE, dtype=CDTYPE)
mask = get_moment_mask(cn)
for it in range(n_iters):
    composed = geometry.synthesise_series(cn, PolynomialFamily)
    composed[:, n_known_betas:] *= 0
    print("it:", it, "MSE:", MSE(composed, la_sinos))
    cn[:, mask] += geometry.series_expand(la_sinos-composed)[:, mask]

sinos computed
it: 0 MSE: tensor(142.3576)


TypeError: FlatFanBeamGeometry.series_expand() missing 2 required positional arguments: 'PolynomialBasis' and 'n_degs'

In [ ]:
phantoms = get_htc2022_train_phantoms()
print("phantoms loaded")
sinos = geometry.project_forward(phantoms)
la_sinos, known_angles = geometry.zero_cropp_sinos(sinos, ar, 0)

N, Nu, Nb = sinos.shape
n_known_u = geometry.n_known_projections(ar)

mask = get_moment_mask(torch.zeros((1,M,K)))
n_coeffs = mask.sum()
print(n_coeffs, M*(M+1)//2)
coefficients = torch.zeros((N, n_coeffs), dtype=CDTYPE, requires_grad=False)

l1 = 1e-3

n_iters = 100

for it in range(n_iters):

    embedding = torch.zeros((N, M, K), dtype=CDTYPE)
    embedding[:, mask] += coefficients
    res = geometry.synthesise_series(embedding, Legendre)
    loss = MSE(res[:, :n_known_u], la_sinos[:, :n_known_u]) #+ l1*torch.mean(torch.abs(coefficients)**2)

    err = la_sinos - res
    err[:, n_known_u:] *= 0
    coefficients += geometry.series_expand(err, Legendre, M, K)[:, mask]
    print("iter:", it, "loss:", loss.item())

# exp = la_sinos + 0
embedding = torch.zeros((N, M, K), dtype=CDTYPE)
embedding[:, mask] += coefficients
exp = geometry.synthesise_series(embedding, Legendre)
recons = geometry.fbp_reconstruct(exp)
print("exp error:", MSE(exp, sinos))
print("recons error:", MSE(recons, phantoms))

disp_ind = 0
plt.subplot(121)
plt.imshow(sinos[disp_ind])
plt.subplot(122)
plt.title("exp")
plt.imshow(exp[disp_ind].detach())

plt.figure()
plt.subplot(121)
plt.imshow(recons[disp_ind])
plt.subplot(122)
plt.title("gt")
plt.imshow(phantoms[disp_ind])

plt.show()

for i in plt.get_fignums():
    fig = plt.figure(i)
    title = fig._suptitle.get_text() if fig._suptitle is not None else f"fig{i}"
    plt.savefig(f"{title}.png")





